# 基础概念

只有当证券价格是均值回归或者是趋势的，交易策略才能盈利，否则价格是随机游走的，交易将无法盈利。

证券价格一般而言是非常接近随机游走的，但并不意味着在特殊条件下价格不会表现出一定程度的均值回归或趋势行为。构建一个策略，实际上就是要判断在特定的条件和特定的时间段，价格究竟是均值回归的还是趋势的。

价格在同一时间既均值回归又趋势的现象，被有些人描述为价格具有“分形”性质。一个安全的假设是：除非公司的预期盈利发生了变化，股票价格会均值回归。寻找够强够长的均值回归，以保证在引入交易成本后仍有利可图，则是交易员的任务。（\cite{Chan:2009}）

过度竞争对策略的影响：
* 对于均值回归策略，竞争的结果就是套利机会的消失，使得收益率逐渐降低为 0.
* 对于趋势策略，竞争的结果是减少该趋势持续的时间周期。

# 均值回归策略（Mean-Reverting Strategy）

均值回归分为：
* 时间序列均值回归：证券的价格回归到自身的时间序列均值。
* 横截面均值回归：一篮子证券的累积收益率回归到整个篮子累积收益水平，换句话说，证券的相对收益在时间序列上是负相关的。


## 平稳性（Stationarity）

简单讲，一个时间序列不会越来越大的偏离初始值就是平稳的。

大多数的证券价格序列都是不平稳的，主要表现为几何随机游走。然而收益率序列通常围绕 0 上下波动，但收益率的均值回归不能形成交易策略。

如果一个价格序列是平稳的，只要未来继续保持平稳，采用均值回归策略一定能盈利。但成功的均值回归策略并不一定要求平稳的价格序列。即使非平稳的序列也可能有很多可以利用的短期回归机会。

均值回归和平稳性是对时间序列相同性质的等价描述，但引出了对平稳性的两种统计检验方法。均值回归是指下一期价格的变化正比于当前价格和均值的偏离，这和 ADF 检验的方法一致，ADF 检验的原假设是该比例系数为 0。平稳时间序列其对数序列的方差的增长率要小于几何随机游走，随机游走的方差是时间的线性函数，从而平稳序列的对数序列的方差是时间的次线性（sublinear）函数。这通常用 Hurst 指数描述，方差比检验（Variance Ratio Test）就是去检验是否可以拒绝序列的 Hurst 指数为 0.5（几何随机游走的 Hurst 指数为 0.5）的原假设。

### ADF 检验（Augmented Dickey-Fuller Test）

价格的变化服从线性模型：
$$
\Delta y_t = \lambda y_{t-1}+\mu+\beta t+\alpha_1\Delta y_{t-1}+\cdots+\alpha_k\Delta y_{t-k}+\varepsilon_t
$$

ADF 检验的原假设是 $\lambda=0$，如果拒绝原假设，就说明价格变化依赖于当前的价格水平，从而其并非随机游走。另外，ADF 检验还有三个参数：
* $\beta$：价格序列关于时间的漂移项相对于价格的波动非常微小，简单起见可以假设 $\beta=0$；
* $\mu$：长期价格均值通常不为 0，即 $\mu\neq0$；
* $k$：可以从 $k=0$ 开始尝试，通常对于 $k=1$ 可以拒绝原假设，这说明价格的变化有一定的序列相关性。

### 方差比检验（Variance Ratio Test）

直观地讲，平稳时间序列意味着序列从初始值开始扩散的速度比几何随机游走慢。从数学上可以通过测量扩散速度来考察时间序列，扩散速度可以用下面的方差描述：
$$
Var(\tau) = \operatorname{Avg}\left[|\log y_{t+\tau}-\log y_t|^2\right]
$$
其中，均值是关于 $t$ 取的。$Var(\tau)$ 关于 $\tau$ 的幂次有如下形式：
$$
Var(\tau) \sim \tau^{2H}
$$
$H$ 称为 Hurst 指数，对于几何随机游走，有 $H=0.5$，对于均值回归序列，有 $H<0.5$，对于趋势序列，有 $H>0.5$。H 越小，序列越倾向于均值回归；H 越大，序列越倾向于趋势。H 可以作为均值回归程度或者趋势程度的衡量指标。

方差比检验用于检验 Hurst 指数偏离 0.5 的显著性，原假设是 $H=0.5$。方差比检验的统计量为：
$$
\frac{\operatorname{Var}\left[\log y_t-\log y_{t-\tau}\right]}{\tau\operatorname{Var}\left[\log y_t-\log y_{t-1}\right]}
$$
方差比检验该统计量是否等于 1。


### 均值回归的速度

ADF 检验中的参数 $\lambda$ 可以理解为均值回归的速度。如果以连续的 Ornstein-Uhlenbeck 过程来对均值回归序列建模：
$$
{\rm d}z_t = \lambda(z_t - \mu){\rm d}t+\sigma{\rm d}W_t
$$
其中，$\mu$ 是长期均值，$\lambda<0$ 是均值回归的速度，$W_t$ 是噪声项，比如布朗运动。$z_t$ 的期望为：
$$
\mathbb{E}[z_t] = z_0e^{\lambda t} + \mu\left(1-e^{\lambda t}\right)
$$
是指数衰减过程，其半衰期为 $\frac{\log2}{|\lambda|}$。可以用 ${\rm d}z_t$ 对于 $z_t$ 进行回归得到 $\mu,\lambda$ 的估计值。

如果 $\lambda$ 非常接近于 0，则说明半衰期很长，均值回归的速度很小，据此构建的均值回归策略盈利性较差。另外，半衰期也可用于确定策略参数计算的回溯期。


## 协整性（Cointegration）

协整性是指若干非平稳序列存在线性组合是平稳的。

两个价格序列的相关性和协整性并不相同，相关性指在一段时间内其收益率的相关性，正相关表示两个序列的价格变化方向在大多数时间是相同的，但并不反映两个序列的长期行为特征，无法保证两个序列在长期内价格偏离不会越来越大。协整性在会保证两个序列不大可能会偏离太大，即使它们的收益率可能不相关。

### Engle-Granger 协整性检验

1. 变量的非平稳性检验，确认变量的单整阶数相同；
2. 对变量进行 OLS 回归，得到残差序列，用 ADF 方法检验残差序列的平稳性；

理论上 n 个变量都是相同阶数的单整过程，那么这些变量之间最多存在 n-1 个协整关系。Engle-Granger 检验的缺陷是其只能识别出一个协整关系。


### Johansen 检验

记 $\mathbf{Y}_t=(y_{t,1},y_{t,2},\ldots,y_{n,t})$ 是 n 个时间序列，考察方程：
$$
\Delta \mathbf{Y}_t=\Lambda\mathbf{Y}_{t-1}+\mathbf{\mu}+\mathbf{A}_1\Delta\mathbf{Y}_{t-1}+\cdots+\mathbf{A}_k\Delta\mathbf{Y}_{t-k}+\mathbf{\varepsilon}_t
$$
如果矩阵$\Lambda$的秩为 r，那么这 n 个时间序列之间存在 r 个协整关系，对应的特征向量为协整关系式的系数，或者组合的对冲比例。


## 动态对冲比例

对于真正协整的价格序列，对冲比例的确定非常简单，收集尽可能充足和历史样本，通过线性回归或者 Johansen 检验的特征向量即可确定对冲比例。然而现实中，真正的协整很难满足，或者协整关系和对冲比例会随着时间而变化。

一个简单的对冲比例的动态调整方法是选定一个回溯期，在每个时点用回溯期内的历史样本进行线性回归或者 Johansen 检验的特征向量来得到当前的对冲比例。这个方法的缺点是：如果回溯期较短，则回溯窗口移动时会造成对冲比例的剧烈变化；回溯期过长则可能无法捕捉到价格序列之间的最新关系。本质的问题是回溯期长度不宜确定。另外，可以通过对样本赋权的方式改善对冲比例的估计，比如以指数加权的方式对回溯期内的样本点赋权。


### 卡尔曼滤波（Kalman Filter）

卡尔曼滤波是基于最新的观测变量（Observable Variable）更新隐含变量（Hidden Variable）的最优线性算法。线性是指它假设观测变量是隐含变量的线性函数。最优是指在假设噪声分布是正态的条件下，该算法最小化均方误差。

\cite{Chan:2013} 给出了利用卡尔曼滤波估计对冲比例的模型：

假设有两个价格序列 $y_t, x_t$, 试图估计 $x_t$ 关于 $y_t$ 的对冲比例 $\beta_t$，设 $y_t$ 为观测变量，而对冲比例 $\beta_t$ 为隐含变量，则测量方程（Measurement Equation）为：
$$
y_t = \alpha_t+\beta_t \cdot x_t + \varepsilon_t
$$
其中，$\varepsilon_t$ 为高斯白噪声，其方差设为 $V_\varepsilon$。$\alpha_t,\beta_t$ 均是要估计的隐含变量。这里记 $\mathbf{b}_t=(\alpha_t, \beta_t)^T$，$\mathbf{x}_t=(1, x_t)^T$。则测量方程为：
$$
y_t = \mathbf{b}^T_t\cdot\mathbf{x}_t + \varepsilon_t
$$

对状态转移的一个简单假设是期望对冲比例不变，即状态转移方程（State Transition Equation）为：
$$
\mathbf{b}_t = \mathbf{b}_{t-1}+\mathbf{w}_{t}
$$
其中，$\mathbf{w}_{t}$ 为高斯白噪声，其协方差矩阵设为 $V_\mathbf{w}$。

对观测变量的估计可以通过一个迭代算法实现，该估计值就可以类似作为移动平均值使用，算法给出的观测变量的预测误差就可以类似作为移动标准差使用。

记给定 $t-1$ 时刻观测值对 $t$ 时刻隐含变量的估计为 $\hat{\mathbf{b}}_{t|t-1}$，对观测变量的估计为 $\hat{y}_{t|t-1}$，给定 $t$ 时刻观测值对 $t$ 时刻隐含变量的估计为 $\hat{\mathbf{b}}_{t|t}$。假设已知 $t-1$ 时刻隐含变量的估计值 $\hat{\mathbf{b}}_{t-1|t-1}$ 以及隐含变量方差$\operatorname{cov}\left[\mathbf{b}_{t-1}-\hat{\mathbf{b}}_{t-1|t-1}\right]$ 的估计 $\hat{\mathbf{R}}_{t-1|t-1}$，则在时刻 $t$，有：
1. 状态预测：$\hat{\mathbf{b}}_{t|t-1} = \hat{\mathbf{b}}_{t-1|t-1}$
2. 状态方差预测：$\hat{\mathbf{R}}_{t|t-1} = \hat{\mathbf{R}}_{t-1|t-1}+V_\mathbf{w}$
3. 观测变量预测：$\hat{y}_t=\hat{\mathbf{b}}_{t|t-1}^T\cdot\mathbf{x}_t$
4. 观测变量方差预测：$\hat{Q}_t=\mathbf{x}_t^T\cdot\hat{\mathbf{R}}_{t|t-1}\cdot\mathbf{x}_t+V_\varepsilon$
5. 状态跟新：$\hat{\mathbf{b}}_{t|t} = \hat{\mathbf{b}}_{t|t-1}+\mathbf{k}_t\cdot e_t$
6. 状态方差更新：$\hat{\mathbf{R}}_{t|t} = \hat{\mathbf{R}}_{t|t-1} - \mathbf{k}_t\cdot\mathbf{x}^T_t\cdot\hat{\mathbf{R}}_{t|t-1}$

其中，$\mathbf{k}_t = \hat{\mathbf{R}}_{t|t-1}\cdot\mathbf{x}_t/\hat{Q}_t$，$e_t=y_t-\hat{\mathbf{b}}_{t|t-1}^T\cdot\mathbf{x}_t$。另外，对于迭代初始值，可以令 $\hat{\mathbf{b}}_{0|0} = \mathbf{0}，\hat{\mathbf{R}}_{0|0} = \mathbf{0}$。滤波参数 $V_\varepsilon, V_\omega$ 的估计参考 \cite{Rajamani:2009} 和 \cite{Rajamani:2007}


## 交易标的

### 价差序列

如果证券的价格序列 $(y_{t,1}, y_{t,2}, \ldots, y_{t,n})$ 有下面的协整关系：
$$
y_t = h_1y_{t,1}+h_2y_{t,2}+\cdots+h_ny_{t,n}
$$
则可以通过交易组合的价格序列 $y_t$ 构建均值回归策略，其中，$h_i$ 为第 i 个证券的投资数量。

### 对数价差序列

如果证券的对数价格序列 $(\log y_{t,1}, \log y_{t,2}, \ldots, \log y_{t,n})$ 有下面的协整关系：
$$
\log y_t = h_1\log y_{t,1}+h_2\log y_{t,2}+\cdots+h_n\log y_{t,n}
$$
则可以通过交易组合的价格序列 $y_t$ 构建均值回归策略，其中，$h_i$ 为第 i 个证券的投资金额比例。这是因为对上式做一阶差分：
$$
\Delta\log y_t = h_1\Delta\log y_{t,1}+h_2\Delta\log y_{t,2}+\cdots+h_n\Delta\log y_{t,n}
$$
对数价格的变化就是收益率（$\Delta\log x\approx\Delta x/x$），上式表明组合的收益率等于各成份证券收益率的线性组合，即 $h_i$ 代表的是配置比例。

### 价格比率序列

当考察的两个价格序列并不真正协整，但从其他方面出发有理由认为两个序列有一定的均值回归效应，可以考虑两个序列的比值：
$$
y_t = \frac{y_{t,1}}{y_{t,2}}
$$
如果出现交易信号，以 1:1 的资金比例构建组合。


## 策略构建

实际上，构建一个有效的均值回归策略并不需要真正的平稳性或者协整性（比如捕捉一些短期的或者特定条件下的均值回归），而且也并不是所有的平稳序列都能构建有效的策略（比如其均值回归的速度过慢）。

简单的均值回归策略如下构建：
1. 通过回归或者协整检验确定对冲比例，构建组合价格序列；
2. 计算回溯期内组合价格序列的均值和标准差，并进而通过 z-score 方法标准化组合价格序列；
3. 如果当期组合价格小于 0，则做多组合；如果大于 0，则做空组合。整体的投资规模（数量或者金额）取决于偏离程度。


### 布林带（Bollinger Band）策略

1. 通过回归或者协整检验确定对冲比例，构建组合价格序列；
2. 计算回溯期内组合价格序列的均值 $\mu_t$ 和标准差 $\sigma_t$；
3. 给定建仓的标准差倍数 $M_{entry}$，如果当期组合价格高于均值的幅度超过 $M_{entry}\sigma_t$，则做空组合；如果当期组合价格低于均值的幅度超过 $M_{entry}\sigma_t$，则做多组合；
4. 给定平仓的标准差倍数 $M_{exit}$，如果当期组合价格偏离均值的幅度小于 $M_{exit}\sigma_t$，则平仓。


## 风险管理

当均值回归策略出现较大的回撤时，通常是因为基本面的原因，这很难在事前判断。均值回归策略的风险管理非常重要而困难，一般的止损方式很难应用于均值回归策略。


# 趋势策略（Momentum Strategy）

又称为动量策略。趋势存在于各种资产中, 自 Jegadeesh 和 Titman 首先在 1993 年 Journal of Finance 上发表了动量因子(Momentum Factor)的研究成果之后, 关于趋势本身以及其原因的探讨从未停止过, 趋势广泛存在至少有以下原因:
* 基本面信息, 尤其是政策信息具有延续性：资产价格的长期趋势通常与基本面信息高度相关, 而基本面很大程度会受到政府政策等因素影响, 如政府在利率上的政策会影响外汇、利率以及股票等资产的长期趋势, 由于通常政府的政策具有延续性, 从而资产价格的延续性便很容易被投资者接受.
* 市场非有效, 信息的缓慢扩散：通常认为资本市场参与者按照拥有信息的多寡和质量分为三种: 知情交易者、非知情交易者以及噪音交易者. 知情交易者会提前知道市场信息而提前进入市场进行布局, 随着信息的扩散, 越来越多的非知情交易者得知该信息会随后进入市场, 使得资产价格达到“合理价格”需要一定的时间, 于是在价格表现上行情会有延续。
* 各种类型基金的强制被动买卖行为。
* 对于期货，展期收益率（主要是其符号）的持续性。
* 高频交易引起的市场操纵。
* 流动性需求：大额订单分拆执行，这可能导致更多的短期趋势。
* 投资者的羊群行为：投资者把其他人的买卖决策（可能是随机且无意义的）作为自己交易决策的唯一判断标准。

流动性需求和羊群效应产生的趋势的时间周期很难预测，如果没有估计时间周期的可靠方法，就无法通过趋势交易获利。

趋势分为：
* 时间序列趋势（动量）：证券过去的收益率与未来收益率正相关。
* 横截面趋势（动量）：过去相对收益高的证券未来的相对收益依然较高。

相比于均值回归策略，一般来说获得夏普比率较高的趋势策略相对困难：（\cite{Chan:2013}）
* 大多数日级别的趋势策略其持有期较长，交易频率较低，从而夏普比率不高。
* **动量崩溃（Momentum Crash）**现象：动量策略在金融危机后倾向于失效。（\cite{Daniel:2016}）
* 当同质策略竞争剧烈时，趋势的持续时间急剧缩短。


## 相关性检验

对于序列$\{x_i\}_{i=1}^n,\{y_i\}_{i=1}^n$，其 Pearson 相关系数：
$$
r = \frac{\operatorname{Cov}(x,y)}{\sqrt{\operatorname{Var}(x)\cdot\operatorname{Var}(y)}}
$$

相关性检验用于检验相关系数是否显著不为 0，原假设是 $r=0$，备择假设为 $r\neq0$。相关性检验的统计量为：
$$
T = \frac{r\sqrt{n-2}}{\sqrt{1-r^2}}\sim t(n-2)
$$

在对时间序列进行相关性检验时，需要区分回溯期（Look-back Period）和持有期（Holding Period），并且寻找正相关的显著性最高的回溯期和持有期。另外除了检验收益率的相关性，还可以检验收益率符号的相关性。

在计算回溯期收益率序列和持有期收益率序列时需要注意不能有重叠，即当持有期短于回溯期时，持有期不能重叠；当回溯期短于持有期时，回溯期不能有重叠。

![回溯期和持有期](.\images\回溯期和持有期.PNG)

如果有多个“回溯期-持有期”组合的收益率是显著正相关的，持有期较短的更好，可以增加交易次数，从而增大夏普比率。


## 方差比检验（Variance Ratio Test）

如果不考虑特定的时间片，而只是考察序列长期的趋势性，则类似于均值回归策略，方差比检验也可以用于检测趋势性。如果 Hurst 指数显著大于 0.5，则表明序列有长期的趋势性。

## 策略构建

趋势策略的本质是进行趋势跟随，即当资产价格趋势向上时候买入，反之卖出。

### 趋势确定

确定当前趋势方向方法和指标有很多，常见的有：

**收益率**：过去一段时间的收益率为正则做多，为负则做空。

**移动平均线**：当前价格高于均线则做多，反之则做空。

**回归法**：因变量是资产价格，自变量是时间轴，趋势确定由回归系数的正负或者显著性确定。


### 过滤器

为了尽量避免陷入虚假趋势中，可以通过添加过滤器来提升趋势策略的效果。过滤器用于过滤趋势不明显的信号，即只有当趋势显著才认为是有效的开仓信号。
过滤器的作用是降低波动和回撤，降低交易次数并提高胜率，当然也会牺牲一定的收益率。

常见的过滤器有：

**Bolling Band**：价格走势突破上轨则做多，突破下轨则做空。

**均值回复过滤器**：如果确定当前趋势向上，但同时要求当前价格低于最近一段时间的价格（短期收益率为负），即要求长期来看是趋势但短期回调；反之亦然。

**Alexander 过滤器**：

**价格效率**：
$$
\rm{Efficiency}_t = \frac{P_t-P_{t-L}}{\sum\limits_{\tau=1}^{L}\left|P_{t-\tau+1} - P_{t-\tau}\right|}
$$
其中，$P_t$ 为 t 时点的价格。确定当前趋势向上，同时要求价格效率较大。

**趋势强度**
$$
\rm{Trend Strength}_t = \left|\frac{P_t-P_{t-L}}{P_{t-L}}\right|\Bigg/\frac{\max\limits_{0\leq\tau\leq L}\{H_t-\tau\}-\min\limits_{0\leq\tau\leq L}\{L_t-\tau\}}{\min\limits_{0\leq\tau\leq L}\{L_t-\tau\}}
$$
其中，$H_t,L_t$ 分别为 t 时点的最高价和最低价。趋势强度指标越大，表明当前趋势越强，反之震荡性更强。


### 止损


## 多因子模型（Multi-Factor Model）

多因子模型将证券的超额收益率分解成系统的因子收益率和特异性收益率。

选择的因子的合理性取决于因子模型能否充分的解释证券的超额收益率，否则回归将产生显著的特异性收益率，即拟合方程的 $R^2$ 将会很小。比如通常的经验规律是 1000 只股票的月度收益率和 50 个因子之间好的因子模型，其 $R^2$ 应当在 30%~40% 之间。

多因子策略盈利的假设前提是因子收益率比个股收益率更稳定，即**因子收益率具有趋势（动量）**，因此可以假设它们的值从当期到下一期保持不变，再加上特异性收益率不显著，则可以据此预测超额收益率。从而，基本面和宏观因子主导的因子模型有效的前提就是投资者始终使用相同的标准对公司进行估值。在投资者的估值标准（因子收益率）发生改变的这段时间内，多因子策略很可能出现大幅回撤，即使是很短的一段时间。然而，对于任何隔夜持仓的策略，这是个常见问题。

# 状态转换（Regime Switching）

寻找状态转换即是寻找所谓的“拐点”，这种非常困难。常见的状态转换包括：
* 经济周期
* 波动率高低
* 均值回归与趋势（震荡市与牛熊市）

这其中波动率状态转换似乎最适用经典计量经济学工具。

## Markov 状态转换模型

又称作隐 Markov 模型（Hidden Markov Model）：
* 假设价格在两个或多个状态上的概率分布不同，比如两个状态均服从对数正态分布，但均值和标准差不同
* 假设状态之间存在某种转移概率
* 使用诸如极大似然估计的标准统计方法，通过拟合历史数据，来确定各个状态的概率分布和转移概率

Markov 状态转换模型在实际应用中的缺陷：模型假设状态之间的转移概率固定，这导致证券从正常的静止状态转移到不稳定状态的概率非常小。

## 拐点模型（Turning Point Model）

使用了数据挖掘方法，输入所有可能的标量来预测拐点或状态转换。


# 平仓信号

每个策略都有特定的建仓信号，但平仓信号主要有以下几种：
* 固定持有期
* 止盈价格或者收益率
* 止损价格或者收益率
* 新的建仓信号

对于均值回归模型，前面给出了确定其回归半衰期的方法，半衰期可以用来确定最优持有期。另外，其均值水平也可以作为一个目标平仓价格（止盈价格）。两个条件满足其中一个就可以平仓。对于均值回归模型，基于持有期或者止盈价格平仓比止损平仓更加合理，因为止损意味着在更加极端的情况下平仓，这与模型的假设相违背。除非可以确定由于新信息的影响，市场已经由均值回归状态转变为趋势状态。

趋势策略的最优持有期会随着信息扩散的速度加快以及使用同一策略的人数增多而缩短。由于历史数据有限，回测时难免有过拟合偏差。趋势策略可以通过不断用新的数据运行模型，直到得到相反信号时来选择平仓或者反向建仓。这就是以最新的建仓信号作为平仓信号。这其实也是一个止损的情形，因为如果模型最新的信号与当前头寸方向相反，趋势的方向很可能已经改变，头寸已经发生了回撤，此时清仓相当于止损，这显然比人为添加止损参数来得更合理，因为引入参数的过程难免会有过拟合偏差。

# 季节性策略（Seasonal Strategy）

季节性策略利用的是证券市场上的日历效应（Calendar Effect）。这种策略建议在某个固定的日期或者时间买入或者卖出某种证券，并在另一个固定的日期或者时间平仓。常见的日历效应：
* 一月效应（January Effect）：前一年收益率最差的小盘股，在本年一月的收益率普遍高于前一年收益最好的那些小盘股。（\cite{Singal:2006}，\cite{Chan:2009}）
* 同月反转效应：每个月去年同月收益最差的股票表现将由于去年同月收益最好的股票。（\cite{Heston:2008}，\cite{Chan:2009}）

商品期货市场的季节性策略相比股票市场有效性更好，可能是因为对某种商品的需求来源于实体经济而非投机。季节性策略的缺陷是通常一年只能交易很少的次数，从而无法判断回测结果是否由于过拟合偏差引起。减少这一问题的方法之一是通过选择不同的建仓和平仓时间来考察盈利能力是否稳定。另外，通常的原则是交易那些有实际经济意义的季节性策略。

# 高频策略（High-Frequency Strategy）

由于流动性的原因，高频策略最早出现于外汇市场，随后是期货市场，后面才是股票等市场。

高频交易策略能获得高 Sharpe 比率的理由是因为根据大数定律交易的次数越多，实际收益率相对于期望收益率偏差就越小。如果一个策略有正的期望收益率，高频交易获得的收益率相对于期望收益的偏差将最小。高频策略的高 Sharpe 比率可以使用更高的杠杆水平以大大提高策略的净值收益率。

通常来讲，高频策略要么是利用市场中出现的微小无效性获利，要么是通过提供短暂的流动性以获取微小的报酬。与基于宏观趋势或者公司基本面的策略会时常经历大的波动不同，高频交易策略依赖于每天都会出现的市场短暂无效和流动性需求，这使得它可以每天持续盈利。另外，高频交易的交易量适中，因此风险管理比较容易：亏损时可以很快地“去杠杆”，市场条件不合适时可以随时停止交易。最坏的结果只是收益率不断下滑而最终无利可图。突发大额亏损则不太可能。

高频交易对交易成本比较敏感。

# References

[<a id="cit-Chan:2009" href="#call-Chan:2009">Chan:2009</a>] Ernest Chan, ``_Quantitative trading: how to build your own algorithmic trading business_'',  2009.

[<a id="cit-Chan:2013" href="#call-Chan:2013">Chan:2013</a>] Ernest Chan, ``_Algorithmic trading: winning strategies and their rationale_'',  2013.

[<a id="cit-Rajamani:2009" href="#call-Rajamani:2009">Rajamani:2009</a>] Murali Rajamani and James Rawlings, ``_Estimation of the Disturbance Structure from Data using Semidefinite Programming and Optimal Weighting_'', Automatica, vol. 45, number , pp. 142--148,  2009.

[<a id="cit-Rajamani:2007" href="#call-Rajamani:2007">Rajamani:2007</a>] Murali Rajamani, ``_Data-based Techniques to Improve State Estimation in Model Predictive Control_'',  2007.

[<a id="cit-Daniel:2016" href="#call-Daniel:2016">Daniel:2016</a>] Kent Daniel and Tobias Moskowitz, ``_Momentum Crashes_'', Journal of Financial Economics, vol. 122, number 2, pp. 221--247,  2016.

[<a id="cit-Singal:2006" href="#call-Singal:2006">Singal:2006</a>] Vijay Singal, ``_Beyond the Random Walk: A Guide to Stock Market Anomalies and Low-Risk Investing_'',  2006.

[<a id="cit-Heston:2008" href="#call-Heston:2008">Heston:2008</a>] Steven Heston and Ronnie Sadka, ``_Seasonality in the Cross-Section of Expected Stock Returns_'', Journal of Financial Economics, vol. 87, number 2, pp. 418--445, February 2008.  [online](https://lcb1.uoregon.edu/rcg/seminars/seasonal072604.pdf)

